# 05 - 進階中介層 (Advanced Middleware)

## 📚 學習目標

- ✅ 理解中介層 (Middleware) 的作用
- ✅ 使用 @before_model 裝飾器
- ✅ 使用 @after_model 裝飾器
- ✅ 使用 @wrap_tool_call 裝飾器
- ✅ 實作日誌、驗證、快取等中介層功能

## ⏰ 預計時間: 40 分鐘

## 📦 環境設定

In [ ]:
!pip install langchain==0.1.0
!pip install langchain-openai==0.0.5
!pip install python-dotenv==1.0.0

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
print("✅ 環境設定完成")

## Part 1: 什麼是 Middleware?

### 1.1 Middleware 概念

Middleware (中介層) 是在主要邏輯執行前後插入的處理邏輯:

```
┌──────────────────────────────────────┐
│         Middleware 執行流程           │
├──────────────────────────────────────┤
│                                      │
│  請求 → @before_model                │
│            ↓                         │
│         主要邏輯 (LLM/Tool)          │
│            ↓                         │
│        @after_model → 回應           │
│                                      │
└──────────────────────────────────────┘
```

### 常見用途
- 📝 日誌記錄
- ✅ 輸入驗證
- 💾 快取管理
- 🔒 權限檢查
- ⏱️ 效能監控
- 🔄 重試邏輯

## Part 2: @before_model 裝飾器

### 2.1 基礎用法

In [ ]:
from functools import wraps
from datetime import datetime

def before_model(func):
    """在模型執行前處理"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"\n⏰ [{datetime.now().strftime('%H:%M:%S')}] 開始執行")
        print(f"📥 輸入: {args[0] if args else kwargs}")
        
        # 執行原函數
        result = func(*args, **kwargs)
        
        return result
    return wrapper

@before_model
def query_policy(policy_id: str) -> str:
    """查詢保單"""
    return f"保單 {policy_id} 的資訊: 終身壽險, 投保人王小明, 保額 200 萬"

# 測試
result = query_policy("CL202401001")
print(f"📤 輸出: {result}")

### 2.2 輸入驗證 Middleware

In [ ]:
import re

def validate_policy_id(func):
    """驗證保單號碼格式"""
    @wraps(func)
    def wrapper(policy_id: str, *args, **kwargs):
        # 驗證格式: CL + 數字
        if not re.match(r'^CL\d{9}$', policy_id):
            raise ValueError(f"❌ 無效的保單號碼格式: {policy_id} (應為 CL + 9 位數字)")
        
        print(f"✅ 保單號碼驗證通過: {policy_id}")
        return func(policy_id, *args, **kwargs)
    return wrapper

@validate_policy_id
def get_policy_info(policy_id: str) -> str:
    return f"查詢保單 {policy_id} 成功"

# 測試有效輸入
print("\n📋 測試 1: 有效保單號碼")
result = get_policy_info("CL202401001")
print(result)

# 測試無效輸入
print("\n📋 測試 2: 無效保單號碼")
try:
    result = get_policy_info("INVALID")
except ValueError as e:
    print(e)

### 2.3 權限檢查 Middleware

In [ ]:
# 模擬使用者權限資料庫
USER_PERMISSIONS = {
    "agent001": ["query", "update"],
    "agent002": ["query"],
    "admin": ["query", "update", "delete"]
}

def require_permission(required_permission: str):
    """檢查使用者權限"""
    def decorator(func):
        @wraps(func)
        def wrapper(user_id: str, *args, **kwargs):
            # 檢查權限
            user_perms = USER_PERMISSIONS.get(user_id, [])
            
            if required_permission not in user_perms:
                raise PermissionError(f"❌ 使用者 {user_id} 無 '{required_permission}' 權限")
            
            print(f"✅ 權限檢查通過: {user_id} 擁有 '{required_permission}' 權限")
            return func(user_id, *args, **kwargs)
        return wrapper
    return decorator

@require_permission("update")
def update_policy(user_id: str, policy_id: str, data: dict) -> str:
    return f"使用者 {user_id} 成功更新保單 {policy_id}"

# 測試
print("📋 測試 1: 有權限的使用者")
result = update_policy("agent001", "CL202401001", {"status": "active"})
print(result)

print("\n📋 測試 2: 無權限的使用者")
try:
    result = update_policy("agent002", "CL202401001", {"status": "active"})
except PermissionError as e:
    print(e)

## Part 3: @after_model 裝飾器

### 3.1 結果處理

In [ ]:
def after_model(func):
    """在模型執行後處理"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        
        print(f"\n⏰ [{datetime.now().strftime('%H:%M:%S')}] 執行完成")
        print(f"📤 輸出: {result}")
        
        return result
    return wrapper

@before_model
@after_model
def calculate_premium(age: int, coverage: int) -> int:
    """計算保費"""
    base_rate = 0.02
    age_factor = 1 + (age - 30) * 0.01
    return int(coverage * base_rate * age_factor)

# 測試
premium = calculate_premium(35, 2000000)
print(f"\n💰 最終保費: ${premium:,}")

### 3.2 結果格式化 Middleware

In [ ]:
from typing import Any

def format_currency(func):
    """將數字結果格式化為貨幣"""
    @wraps(func)
    def wrapper(*args, **kwargs) -> str:
        result = func(*args, **kwargs)
        
        # 格式化為貨幣
        if isinstance(result, (int, float)):
            formatted = f"NT$ {result:,.0f}"
            print(f"💵 格式化: {result} → {formatted}")
            return formatted
        
        return result
    return wrapper

@format_currency
def get_policy_value(policy_id: str) -> int:
    """取得保單價值"""
    # 模擬查詢
    return 2500000

# 測試
value = get_policy_value("CL202401001")
print(f"保單價值: {value}")

### 3.3 日誌記錄 Middleware

In [ ]:
import json
from typing import List

class AuditLogger:
    """稽核日誌記錄器"""
    
    def __init__(self):
        self.logs: List[dict] = []
    
    def log(self, func_name: str, user: str, action: str, details: dict):
        """記錄操作日誌"""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "function": func_name,
            "user": user,
            "action": action,
            "details": details
        }
        self.logs.append(log_entry)
        print(f"📝 記錄日誌: {func_name} - {action}")
    
    def get_logs(self) -> List[dict]:
        return self.logs
    
    def export_logs(self) -> str:
        return json.dumps(self.logs, indent=2, ensure_ascii=False)

# 建立全域日誌記錄器
audit_logger = AuditLogger()

def audit_log(action: str):
    """稽核日誌裝飾器"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # 執行前記錄
            audit_logger.log(
                func_name=func.__name__,
                user=kwargs.get('user_id', 'unknown'),
                action=action,
                details={"args": str(args), "kwargs": str(kwargs)}
            )
            
            # 執行函數
            result = func(*args, **kwargs)
            
            # 執行後記錄
            audit_logger.log(
                func_name=func.__name__,
                user=kwargs.get('user_id', 'unknown'),
                action=f"{action}_completed",
                details={"result": str(result)}
            )
            
            return result
        return wrapper
    return decorator

@audit_log("update_beneficiary")
def update_beneficiary(policy_id: str, beneficiary: str, user_id: str) -> str:
    """更新受益人"""
    return f"已將保單 {policy_id} 的受益人更新為 {beneficiary}"

# 測試
result = update_beneficiary("CL202401001", "王太太", user_id="agent001")
print(f"\n結果: {result}")

# 查看日誌
print("\n📊 稽核日誌:")
print(audit_logger.export_logs())

## Part 4: @wrap_tool_call 裝飾器

### 4.1 工具呼叫包裝

In [ ]:
import time

def wrap_tool_call(func):
    """包裝工具呼叫,加入監控"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        tool_name = func.__name__
        start_time = time.time()
        
        print(f"\n🔧 呼叫工具: {tool_name}")
        print(f"📥 參數: {args}, {kwargs}")
        
        try:
            result = func(*args, **kwargs)
            execution_time = time.time() - start_time
            
            print(f"✅ 執行成功 (耗時: {execution_time:.3f}秒)")
            print(f"📤 結果: {result}")
            
            return result
            
        except Exception as e:
            execution_time = time.time() - start_time
            print(f"❌ 執行失敗 (耗時: {execution_time:.3f}秒)")
            print(f"錯誤: {e}")
            raise
    
    return wrapper

@wrap_tool_call
def search_policy(policy_id: str) -> dict:
    """搜尋保單"""
    time.sleep(0.1)  # 模擬 API 延遲
    return {
        "policy_id": policy_id,
        "type": "終身壽險",
        "status": "active"
    }

# 測試
result = search_policy("CL202401001")

### 4.2 快取 Middleware

In [ ]:
from functools import lru_cache

class CacheMiddleware:
    """快取中介層"""
    
    def __init__(self, ttl: int = 60):
        self.cache = {}
        self.ttl = ttl  # 快取有效時間(秒)
        self.stats = {"hits": 0, "misses": 0}
    
    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # 建立快取鍵
            cache_key = f"{func.__name__}:{str(args)}:{str(kwargs)}"
            
            # 檢查快取
            if cache_key in self.cache:
                cached_time, cached_value = self.cache[cache_key]
                
                # 檢查是否過期
                if time.time() - cached_time < self.ttl:
                    self.stats["hits"] += 1
                    print(f"💾 快取命中 (總命中: {self.stats['hits']})")
                    return cached_value
            
            # 快取未命中,執行函數
            self.stats["misses"] += 1
            print(f"🔍 快取未命中,執行查詢 (總未命中: {self.stats['misses']})")
            
            result = func(*args, **kwargs)
            
            # 存入快取
            self.cache[cache_key] = (time.time(), result)
            
            return result
        return wrapper
    
    def clear(self):
        """清除快取"""
        self.cache.clear()
        self.stats = {"hits": 0, "misses": 0}
        print("🗑️ 快取已清除")

# 建立快取中介層 (60秒 TTL)
cache = CacheMiddleware(ttl=60)

@cache
def expensive_query(policy_id: str) -> dict:
    """昂貴的查詢操作"""
    time.sleep(0.5)  # 模擬慢速查詢
    return {"policy_id": policy_id, "data": "expensive result"}

# 測試快取
print("📋 第一次查詢:")
result1 = expensive_query("CL202401001")

print("\n📋 第二次查詢 (相同參數):")
result2 = expensive_query("CL202401001")

print("\n📋 第三次查詢 (不同參數):")
result3 = expensive_query("CL202401002")

print("\n📊 快取統計:")
print(f"命中: {cache.stats['hits']} 次")
print(f"未命中: {cache.stats['misses']} 次")
print(f"命中率: {cache.stats['hits'] / (cache.stats['hits'] + cache.stats['misses']) * 100:.1f}%")

## Part 5: 組合多個 Middleware

### 5.1 建立完整的保單查詢系統

In [ ]:
# 組合多個 Middleware
@audit_log("query_policy")
@cache
@validate_policy_id
@wrap_tool_call
def query_complete_policy(policy_id: str, user_id: str = "system") -> dict:
    """完整的保單查詢 (含所有中介層)"""
    time.sleep(0.3)  # 模擬查詢延遲
    
    return {
        "policy_id": policy_id,
        "type": "終身壽險",
        "owner": "王小明",
        "coverage": 2000000,
        "status": "active"
    }

print("="*60)
print("🔍 完整保單查詢測試")
print("="*60)

# 第一次查詢
print("\n📋 測試 1: 第一次查詢")
result1 = query_complete_policy("CL202401001", user_id="agent001")

# 第二次查詢 (應該命中快取)
print("\n📋 測試 2: 第二次查詢 (相同參數)")
result2 = query_complete_policy("CL202401001", user_id="agent001")

# 查看稽核日誌
print("\n" + "="*60)
print("📊 稽核日誌摘要")
print("="*60)
print(f"總操作數: {len(audit_logger.get_logs())}")
print(f"快取命中率: {cache.stats['hits'] / (cache.stats['hits'] + cache.stats['misses']) * 100:.1f}%")

### 5.2 重試 Middleware

In [ ]:
def retry(max_attempts: int = 3, delay: float = 1.0):
    """重試裝飾器"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(1, max_attempts + 1):
                try:
                    print(f"\n🔄 嘗試 {attempt}/{max_attempts}")
                    result = func(*args, **kwargs)
                    print(f"✅ 成功!")
                    return result
                    
                except Exception as e:
                    last_exception = e
                    print(f"❌ 失敗: {e}")
                    
                    if attempt < max_attempts:
                        print(f"⏳ 等待 {delay} 秒後重試...")
                        time.sleep(delay)
            
            # 所有嘗試都失敗
            print(f"\n💥 已達最大重試次數 ({max_attempts}),放棄")
            raise last_exception
        
        return wrapper
    return decorator

# 模擬不穩定的 API
attempt_counter = 0

@retry(max_attempts=3, delay=0.5)
def unstable_api_call(policy_id: str) -> dict:
    """不穩定的 API (前 2 次失敗)"""
    global attempt_counter
    attempt_counter += 1
    
    # 前 2 次失敗
    if attempt_counter < 3:
        raise ConnectionError("API 連線失敗")
    
    # 第 3 次成功
    return {"policy_id": policy_id, "status": "success"}

# 測試
print("🧪 測試重試機制:")
result = unstable_api_call("CL202401001")
print(f"\n🎉 最終結果: {result}")

## 🎯 練習題

### 練習 1: 實作速率限制 Middleware

限制每個使用者每分鐘只能呼叫 10 次

In [ ]:
# TODO: 實作 RateLimitMiddleware

### 練習 2: 實作效能監控 Middleware

記錄每個函數的執行時間統計 (平均、最大、最小)

In [ ]:
# TODO: 實作 PerformanceMonitor

## 📝 學習檢查清單

- [ ] 理解 Middleware 的概念和作用
- [ ] 使用 @before_model 進行前置處理
- [ ] 使用 @after_model 進行後置處理
- [ ] 實作輸入驗證 Middleware
- [ ] 實作權限檢查 Middleware
- [ ] 實作日誌記錄 Middleware
- [ ] 實作快取 Middleware
- [ ] 實作重試 Middleware
- [ ] 組合多個 Middleware
- [ ] 設計完整的中介層系統

## 下一步
繼續前往 `06-langchain-deployment.ipynb` 學習部署實務 👉